# Calculating Cumulative Impact Probability Difference Distribution Between NASA and ESA

In [1]:
# Matplotlib to plot a distribution of cumulative probabilities
import matplotlib.pyplot as plt

# Astropy to read the data and put it in a QTable
import astropy.units as u
from astropy.table import QTable
from astropy.table import join

# Np for data manipulation (np.vectorize etc.)
import numpy as np

In [2]:
# Reading csv file for NASA's data
nasa_table = QTable.read('./NASA_data.csv', 
                         format='ascii.csv',
                         delimiter = ',')
nasa_table[0:3]

des,diameter,fullname,h,id,ip,last_obs,last_obs_jd,n_imp,ps_cum,ps_max,range,ts_max,v_inf
str10,float64,str24,float64,str8,float64,str15,float64,int64,float64,float64,str9,int64,float64
1979 XB,0.66,(1979 XB),18.54,bJ79X00B,8.515158e-07,1979-12-15,2444222.5,4,-2.69,-2.99,2056-2113,0,23.7606234552547
2022 KK2,0.0069,(2022 KK2),28.45,bK22K02K,0.0001203297828,2022-05-23,2459722.5,33,-5.59,-5.79,2060-2122,0,15.5694051293592
2000 SG344,0.037,(2000 SG344),24.79,bK00SY4G,0.002743395186,2000-10-03,2451820.5,300,-2.77,-3.11,2069-2122,0,1.35802744453748


In [3]:
# Checking to see that all of the data from Nasa successfully got loaded
len(nasa_table)

2064

In [4]:
# Reading csv file for ESA's data
esa_table = QTable.read('./ESA_data.csv', 
                         format='ascii.csv',
                         delimiter = ',')
esa_table[0:3]

No.,Object designation,Diameter in m,Impact date in UTC,IP max,PS max,TS,Years,IP cum,PS cum,Vel. in km/s,In list since in d
int64,str15,str11,str10,str9,float64,str3,str9,str9,float64,float64,int64
1,2023VD3,11 - 24*,2034-11-08,1/425,-2.67,0,2034-2039,1/425,-2.67,21.01,825
2,2008JL3,23 - 50*,2027-05-01,1/6711,-2.73,0,2027-2122,1/6211,-2.73,14.01,6328
3,1979XB,400 - 900*,2056-12-12,1/4.27E6,-2.82,0,2056-2113,1/1.36E6,-2.7,27.54,6328


In [5]:
# Checking to see that all of the data from Esa successfully got loaded
len(esa_table)

1906

In [6]:
# Renaming the columns of each table so the the object designation matches (so it can be used as a key) and the cumulative probs specify the dataset
nasa_table.rename_columns(['ip', 'des'], ['IP cum NASA', 'Object Designation'])
esa_table.rename_columns([ 'IP cum', 'Object designation'], ['IP cum ESA', 'Object Designation'])

In [7]:
# Creating a subset of NASA's data with just designation (key) and cumulative probabilities (values)
asteroid_list_nasa = nasa_table['Object Designation', 'IP cum NASA']
asteroid_list_nasa[0:4]

Object Designation,IP cum NASA
str10,float64
1979 XB,8.515158e-07
2022 KK2,0.0001203297828
2000 SG344,0.002743395186
2012 VS76,1.9442009e-05


In [8]:
# The designations for ESA have no spaces, so need to remove them from NASA's data
def remove_spaces(array_w_spaces):
        return array_w_spaces.replace(" ","")

In [9]:
# Vectorizing the function so that it can apply to every entry in the designation column
vremove_spaces = np.vectorize(remove_spaces)

In [10]:
# Removing spaces from NASA's data
asteroid_list_nasa['Object Designation'] = vremove_spaces(asteroid_list_nasa['Object Designation'])
asteroid_list_nasa[0:4]

Object Designation,IP cum NASA
str9,float64
1979XB,8.515158e-07
2022KK2,0.0001203297828
2000SG344,0.002743395186
2012VS76,1.9442009e-05


In [11]:
# Creating a subset of ESA's data with just designation (key) and cumulative probabilities (values)
asteroid_list_esa = esa_table['Object Designation', 'IP cum ESA']
asteroid_list_esa[0:4]

Object Designation,IP cum ESA
str15,str9
2023VD3,1/425
2008JL3,1/6211
1979XB,1/1.36E6
2000SG344,1/354


In [12]:
# The probabilities are fractions displayed as strings, so need to convert them to floats
def convert_frac_to_float(fraction_array):
        try:
            num, denom = fraction_array.split('/')
        except ValueError:
            return None
        return float(num) / float(denom)

In [13]:
# Vectorizing the function so it applies to each entry in the cumulative probability column
vconvert_frac_to_float = np.vectorize(convert_frac_to_float)

In [14]:
# Removing the spaces from ESA just in case, and converting each fraction string into a float
asteroid_list_esa['Object Designation'] = vremove_spaces(asteroid_list_esa['Object Designation'])
asteroid_list_esa['IP cum ESA'] = vconvert_frac_to_float(asteroid_list_esa['IP cum ESA'])
asteroid_list_esa[0:4]

Object Designation,IP cum ESA
str14,float64
2023VD3,0.002352941176470588
2008JL3,0.00016100466913540493
1979XB,7.352941176470589e-07
2000SG344,0.002824858757062147


In [15]:
# Joining the two datasets together by the key Object Designation - if an asteroid only appears in one dataset it does not get put into this joined table
asteroid_common_list = join(asteroid_list_nasa, asteroid_list_esa, keys = 'Object Designation')
asteroid_common_list[0:4]

Object Designation,IP cum NASA,IP cum ESA
str14,float64,float64
1979XB,8.515158e-07,7.352941176470589e-07
1991BA,4.1830306e-06,5.2900256566244344e-06
1993HP1,5.586e-08,2.242152466367713e-07
1993KA2,4.177e-08,3.5211267605633804e-08


In [16]:
# Checking how many asteroids are in both datasets
len(asteroid_common_list)

1724

In [17]:
# This must mean that some asteroids in ESA's dataset aren't in NASA's, even though ESA has less overall asteroids.

In [18]:
asteroid_common_list['IP cum difference'] = asteroid_common_list['IP cum NASA'] - asteroid_common_list['IP cum ESA']
asteroid_common_list['abs val IP cum difference '] = np.abs(asteroid_common_list['IP cum difference'])
asteroid_common_list[0:4]


Object Designation,IP cum NASA,IP cum ESA,IP cum difference,abs val IP cum difference
str14,float64,float64,float64,float64
1979XB,8.515158e-07,7.352941176470589e-07,1.1622168235294108e-07,1.1622168235294108e-07
1991BA,4.1830306e-06,5.2900256566244344e-06,-1.1069950566244348e-06,1.1069950566244348e-06
1993HP1,5.586e-08,2.242152466367713e-07,-1.683552466367713e-07,1.683552466367713e-07
1993KA2,4.177e-08,3.5211267605633804e-08,6.558732394366199e-09,6.558732394366199e-09
